In [1]:
import pandas as pd
import numpy as np
from numba import njit, prange

from tqdm.notebook import tqdm, trange

from numba.typed import List
from numba import types

from numba_progress import ProgressBar

import math

In [2]:
users = 226570
items = 231637

train_data = pd.read_csv("../Preprocessing/processed_dataframes/train.csv")
validation_data = pd.read_csv("../Preprocessing/processed_dataframes/val.csv")

In [3]:
uir_train = train_data.values

uir_val = validation_data.values
n_val = uir_val.shape[0]

In [4]:
@njit
def get_items_rated_by_users(train_data, n_users):
    res = List([List.empty_list(types.int64) for _ in range(n_users)])
    
    for u, i, _ in train_data:
        res[u].append(i)
    
    for u in range(n_users):
        res[u].sort()
    
    return res

irbu = get_items_rated_by_users(uir_train, users)

In [5]:
@njit
def step(
    train_data, 
    Rus, 
    n_users, 
    n_items, 
    k, 
    α1,
    α2,
    α3,
    α4,
    α5,
    λ1, 
    λ2,
    μ, bi, bu, P, Q, Y,
):
    loss = 0
    for u, i, r in train_data:
        Ru = Rus[u]
        sqrt_Ru = np.sqrt(len(Ru))

        implicit_feedback = np.zeros(k)
        for j in Ru:
            implicit_feedback += Y[j]
        implicit_feedback /= (sqrt_Ru+1e-15)

        pred = μ + bu[u] + bi[i] + np.dot(Q[i], P[u] + implicit_feedback)
        error = r - pred

        # Updating
        bu[u] += α1 * (error - λ1*bu[u])
        bi[i] += α2 * (error - λ1*bi[i])

        Pu = P[u]
        Qi = Q[i]
        P[u] += α3*(error*Qi - λ2*Pu)
        Q[i] += α4*(error*(Pu+implicit_feedback) - λ2*Qi)

        term_1 = error*(Qi/(sqrt_Ru+1e-15))
        for j in Ru:
            Y[j] += α5*(term_1 - λ1*Y[j])
            
        loss += error**2
            
    return np.sqrt(loss/len(train_data))

In [6]:
# RS HD page 171 (chrome), 84 book
def fit_svdpp(train_data, val_data, Rus, n_users, n_items, k, α1=.01, α2=.01, α3=.01, α4=.01, α5=.01, λ1=.01, λ2=.01, n_iters=20):
    """
    train_data: array Nx3
    """
    val_ui = uir_val[:, :2]
    val_exp = uir_val[:, -1]
    
    bu = np.zeros(n_users, np.double)
    bi = np.zeros(n_items, np.double)
    
    P = np.random.normal(0, .1, (n_users, k))
    Q = np.random.normal(0, .1, (n_items, k))
    Y = np.random.normal(0, .1, (n_items, k))
    
    μ = np.mean(train_data[:, 2])
    
    model_params = None
    best_epoch = 0
    prev_val_loss = math.inf
    
    t = trange(n_iters, leave=True)
    for it in t:
        loss = step(train_data, Rus, n_users, n_items, k, α1, α2, α3, α4, α5, λ1, λ2, μ, bi, bu, P, Q, Y)
#         α1 *= 0.9
#         α2 *= 0.9
#         α3 *= 0.9
#         α4 *= 0.9
#         α5 *= 0.9
        
        val_preds = predict_batch(val_ui, Rus, (μ, bu, bi, P, Q, Y))
        val_loss = np.sqrt(1/n_val * np.sum((val_preds - val_exp)**2))
        t.set_postfix({"Loss": loss, "Val": val_loss})
        
        if val_loss < prev_val_loss:
            prev_val_loss = val_loss
            model_params = (μ, bu.copy(), bi.copy(), P.copy(), Q.copy(), Y.copy())
            best_epoch = it
    
#     return μ, bu, bi, P, Q, Y
    return model_params

## Fit

In [7]:
@njit
def predict(u, i, Rus, params):
    μ, bu, bi, P, Q, Y = params
    k = P.shape[1]
    
    Ru = Rus[u]
    sqrt_Ru = np.sqrt(len(Ru))

    implicit_feedback = np.zeros(k)
    for j in Ru:
        implicit_feedback += Y[j]
    implicit_feedback /= (sqrt_Ru+1e-15)

    pred = μ + bu[u] + bi[i] + np.dot(Q[i], P[u] + implicit_feedback)
    
    return pred

In [8]:
@njit(parallel=True, nogil=True)
def predict_batch_inner(ui_mat, Rus, params, progress_hook):
    predictions = np.zeros(len(ui_mat))
    for it in prange(ui_mat.shape[0]):
        u, i = ui_mat[it]
        predictions[it] = predict(u, i, Rus, params)
        if np.isnan(predictions[it]):
            print(u, i)
            
        progress_hook.update(1)
        
    return np.clip(predictions, 1., 5.)

def predict_batch_progress_bar(ui_mat, Rus, params):
    with ProgressBar(total=len(ui_mat)) as progress:
        return predict_batch_inner(ui_mat, Rus, params, progress)
    
@njit(parallel=True, nogil=True)
def predict_batch(ui_mat, Rus, params):
    predictions = np.zeros(len(ui_mat))
    for it in prange(ui_mat.shape[0]):
        u, i = ui_mat[it]
        predictions[it] = predict(u, i, Rus, params)
        
    return np.clip(predictions, 1., 5.)

In [9]:
# # α1 = 0.005
# # α2 = 0.005
# # α3 = 0.006
# # α4 = 0.006
# # α5 = 0.006
# # k = 5

# α1 = 0.005
# α2 = 0.005
# α3 = 0.005
# α4 = 0.005
# α5 = 0.005
# λ1 = 0.01
# λ2 = 0.1
# k = 4
# fitted_params = fit_svdpp(
#     uir_train, uir_val, irbu, users, items, k, 
#     α1, α2, α3, α4, α5, λ1, λ2,
#     n_iters=30,
# )

In [10]:
# val_preds = predict_batch(uir_val[:, :2], irbu, fitted_params)
# val_expected = uir_val[:, 2]

# error = np.sqrt(1/n_val * np.sum((val_preds - val_expected)**2))
# print(error)

# ERROR 0.8990364518921785

## Multiple Train

In [11]:
params_product = [
    (0.005, 0.005, 0.01, 0.01, 4),
    (0.005, 0.005, 0.01, 0.01, 5),
    (0.005, 0.005, 0.01, 0.01, 50),
    (0.005, 0.005, 0.01, 0.01, 100),
    (0.005, 0.005, 0.01, 0.1, 4),
    (0.005, 0.005, 0.01, 0.1, 5),
    (0.005, 0.005, 0.01, 0.1, 50),
    (0.005, 0.005, 0.01, 0.1, 100),
    (0.005, 0.005, 0.1, 0.1, 4),
    (0.005, 0.005, 0.1, 0.1, 5),
    (0.005, 0.005, 0.1, 0.1, 50),
    (0.005, 0.005, 0.1, 0.1, 100),
    (0.006, 0.005, 0.01, 0.1, 4),
    (0.006, 0.005, 0.01, 0.1, 5),
    (0.006, 0.005, 0.01, 0.1, 50),
    (0.006, 0.005, 0.01, 0.1, 100),
    (0.005, 0.006, 0.01, 0.1, 4),
    (0.005, 0.006, 0.01, 0.1, 5),
    (0.005, 0.006, 0.01, 0.1, 50),
    (0.005, 0.006, 0.01, 0.1, 100),
]

In [12]:
def train_val(
    uir_train,
    uir_val,
    users,
    movies,
    k,
    α1,
    α2,
    α3,
    α4,
    α5,
    λ1,
    λ2,
    irbu,
    n_iters,
):
    fitted_params = fit_svdpp(
        uir_train, uir_val, irbu, users, items, k, 
        α1, α2, α3, α4, α5, λ1, λ2, n_iters,
    )
    
    val_preds = predict_batch(uir_val[:, :2], irbu, fitted_params)
    val_expected = uir_val[:, 2]
    error = np.sqrt(1/n_val * np.sum((val_preds - val_expected)**2))
    
    return α1, α2, α3, α4, α5, λ1, λ2, k, error

In [13]:
out = [
    train_val(
        uir_train, 
        uir_val,
        users, 
        items,
        k=f, 
        α1=lr1,
        α2=lr1,
        α3=lr2,
        α4=lr2,
        α5=lr2,
        λ1=lamb1, 
        λ2=lamb2,
        irbu=irbu,
        n_iters=23,
    )
    for lr1, lr2, lamb1, lamb2, f in tqdm(params_product)
]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

...

  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

In [14]:
print(sorted(out, key=lambda x: x[-1])[0])

(0.005, 0.005, 0.005, 0.005, 0.005, 0.1, 0.1, 4, 0.8987911144656349)
